**1 Подгрузим данные и необходимые библиотеки**

In [ ]:
# библиотека для скачивания данных с гугл.диска
!pip install --upgrade --no-cache-dir gdown

In [ ]:
# библиотека для векторизации текста, обновим до актуальной версии
!pip install -U gensim

In [ ]:
import gdown
import pandas as pd
import numpy as np
import random
import string
from tqdm import tqdm_notebook as tqdm
import re
from scipy import spatial
import heapq
import timeit
from ast import literal_eval

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
from nltk.stem import SnowballStemmer

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from gensim.models import Word2Vec

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
pd.set_option('display.max_columns', None)

Файлы были взяты из этой задачки: kaggle.com/datasets/rounakbanik/the-movies-dataset

In [ ]:
!gdown 1Co7iQWJhJdqNFTlChd-xT1b88VAzuX4f
!gdown 14OAYTUIw9BoCCSTFIiYzgPq-1XyP0dhQ
!gdown 1hrC11y_r0veFdZqkwH1MSUxSv_CwJtyC

Downloading...
From: https://drive.google.com/uc?id=1Co7iQWJhJdqNFTlChd-xT1b88VAzuX4f
To: /content/movies_metadata.csv
100% 34.4M/34.4M [00:00<00:00, 106MB/s] 
Downloading...
From: https://drive.google.com/uc?id=14OAYTUIw9BoCCSTFIiYzgPq-1XyP0dhQ
To: /content/ratings.csv
100% 710M/710M [00:09<00:00, 72.3MB/s]
Downloading...
From: https://drive.google.com/uc?id=1hrC11y_r0veFdZqkwH1MSUxSv_CwJtyC
To: /content/credits.csv
100% 190M/190M [00:03<00:00, 48.7MB/s]


**2 Проведем первичную обработку данных**

В первом датасете, которым мы воспользуемся, содержится вся основная информация о фильмах, например: бюджет, жанровая принадлежность, язык оригинала, название, описание, дата создания и тд.

In [ ]:
movies_metadata = pd.read_csv('movies_metadata.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
movies_metadata.head(2)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0


In [ ]:
# удалим ошибочные значения
movies_metadata.drop(movies_metadata[movies_metadata['id'] == '1997-08-20'].index, inplace=True)

In [ ]:
movies_metadata.drop(movies_metadata[movies_metadata['id'] == '2012-09-29'].index, inplace=True)

In [ ]:
movies_metadata.drop(movies_metadata[movies_metadata['id'] == '2014-01-01'].index, inplace=True)

В этом наборе мы сосредоточимся на текстовых данных и  будем работать только со стобцами original_title (название фильма) и overview (официальное описание фильма)

In [ ]:
# оставим только колонки, которые мы собираемся использовать
movies_metadata = movies_metadata[['id', 'title', 'overview', 'genres', 'production_companies']]

In [ ]:
# заполняем пустые значения
movies_metadata = movies_metadata.fillna('')

In [ ]:
# удалим дубли, потому что они есть и очень мешаются
movies_metadata = movies_metadata.drop_duplicates().reset_index(drop=True)

In [ ]:
movies_metadata = movies_metadata.rename(columns={'id':'movieId'})

In [ ]:
# меняем тип данных колонки, чтобы проще было джойниться с другой таблицей
movies_metadata['movieId'] = movies_metadata['movieId'].astype(int)

Во втором наборе содержится информация о съемочной команде, нас будет интересовать имя режисера

In [ ]:
credits = pd.read_csv('credits.csv')[['crew', 'id']]

In [ ]:
credits = credits.rename(columns={'id':'movieId'})

In [ ]:
credits = credits.drop_duplicates().reset_index(drop=True)

In [ ]:
credits.head(5)

,crew,movieId
0,"[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862


Теперь поработаем с еще одним набором - пользовательскими оценками фильмов

In [ ]:
ratings = pd.read_csv('ratings.csv')

# выберем только достаточно высоко оцененные фильмы
ratings = ratings[ratings['rating'] > 3]

После подгрузки всех необходимых данных найдем их пересечение по айди фильмов

Дело в том, что не для всех фильмов есть данные о просмотрах и оценках, а для некоторых оценок нет данных о самом фильме

In [ ]:
common_ids = set(movies_metadata.movieId).intersection(set(ratings.movieId))

ratings = ratings[ratings.movieId.isin(common_ids)].reset_index(drop=True)
movies_metadata = movies_metadata[movies_metadata.movieId.isin(common_ids)].reset_index(drop=True)

In [ ]:
movies_metadata.shape[0]

6835

In [ ]:
movies_metadata.groupby('movieId').count().sort_values(by='genres', ascending=False).head(4)

,title,overview,genres,production_companies
movieId,,,,
2,1,1,1,1
65300,1,1,1,1
65216,1,1,1,1
65188,1,1,1,1


In [ ]:
credits = credits[credits.movieId.isin(common_ids)].reset_index(drop=True)

In [ ]:
# здесь мы видим, что размерность не совпадает - есть подозрительные записи
credits.shape[0]

6837

In [ ]:
# ищем задвоенные, но не идентичные записи
credits.groupby('movieId').count().sort_values(by='crew', ascending=False).head(4)

,crew
movieId,
159849,2
4912,2
64650,1
65216,1


In [ ]:
credits[credits['movieId'] == 159849]

,crew,movieId
3181,"[{'credit_id': '52fe4c229251416c910f10d5', 'de...",159849
6056,"[{'credit_id': '52fe4c229251416c910f10d5', 'de...",159849


In [ ]:
credits.drop(credits[credits.index == 3181].index, inplace=True)

In [ ]:
credits[credits['movieId'] == 4912]

,crew,movieId
1525,"[{'credit_id': '52fe43e2c3a36847f80760b5', 'de...",4912
5856,"[{'credit_id': '52fe43e2c3a36847f80760a9', 'de...",4912


In [ ]:
credits.drop(credits[credits.index == 5856].index, inplace=True)

In [ ]:
# убеждаемся, что получили нужный размер
credits.shape[0]

6835

In [ ]:
# соединяем с основным набором
movies_metadata = pd.merge(movies_metadata, credits, on='movieId')

In [ ]:
movies_metadata.head(4) 

,movieId,title,overview,genres,production_companies,crew
0,862,Toy Story,"Led by Woody, Andy's toys live happily in his ...","[{'id': 16, 'name': 'Animation'}, {'id': 35, '...","[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'credit_id': '52fe4284c3a36847f8024f49', 'de..."
1,8844,Jumanji,When siblings Judy and Peter discover an encha...,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...","[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de..."
2,949,Heat,"Obsessive master thief, Neil McCauley leads a ...","[{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...","[{'name': 'Regency Enterprises', 'id': 508}, {...","[{'credit_id': '52fe4292c3a36847f802916d', 'de..."
3,710,GoldenEye,James Bond must unmask the mysterious head of ...,"[{'id': 12, 'name': 'Adventure'}, {'id': 28, '...","[{'name': 'United Artists', 'id': 60}, {'name'...","[{'credit_id': '52fe426ec3a36847f801e14b', 'de..."


Cтолбцы genres, production_companies и crew имеют сложную структуру, но по факту их тип - строка, надо это исправить, чтобы достать из них необходимую нам информацию

In [ ]:
type(movies_metadata['genres'][0])

str

In [ ]:
# literal_eval позволяет восстановить исходную структуру признака из строки
movies_metadata['crew'] = movies_metadata['crew'].apply(literal_eval)
movies_metadata['genres'] = movies_metadata['genres'].apply(literal_eval)
movies_metadata['production_companies'] = movies_metadata['production_companies'].apply(literal_eval)

In [ ]:
# получилось
type(movies_metadata['genres'][0])

list

Напишем и применим функции, достающие имена режисеров, названия студий и жанры фильмов

In [ ]:
def get_director(x):
    for i in x:
        if i['job'] == 'Director':
            return i['name']
    return np.nan

In [ ]:
def get_genres_company(x):
    for i in x:
        return i['name']
    return np.nan

In [ ]:
movies_metadata['director'] = movies_metadata['crew'].apply(get_director)
movies_metadata['genres'] = movies_metadata['genres'].apply(get_genres_company)
movies_metadata['production_company'] = movies_metadata['production_companies'].apply(get_genres_company)

In [ ]:
movies_metadata = movies_metadata[['movieId', 'title', 'overview', 'director', 'genres', 'production_company']]

In [ ]:
movies_metadata.head(4)

,movieId,title,overview,director,genres,production_company
0,862,Toy Story,"Led by Woody, Andy's toys live happily in his ...",John Lasseter,Animation,Pixar Animation Studios
1,8844,Jumanji,When siblings Judy and Peter discover an encha...,Joe Johnston,Adventure,TriStar Pictures
2,949,Heat,"Obsessive master thief, Neil McCauley leads a ...",Michael Mann,Action,Regency Enterprises
3,710,GoldenEye,James Bond must unmask the mysterious head of ...,Martin Campbell,Adventure,United Artists


**3 С помощью обработки текста получим матрицу эмбедингов для каждого фильма**

Нам нужно перевести текстовые данные из названия и описания фильмов в формат, которым может оперировать компьютер - вектора эмбедингов. Перед векторизацией нужно предобработать данные и токенизировать их

После этого мы сможем выполнять операции над этими данными, например сравнивать между собой и находить расстояние между двумя фильмами

In [ ]:
def preprocessing(data, lemmatizer = WordNetLemmatizer(), 
                  stop_words = stopwords.words('english')):
    
    '''
       на вход: 
       - data - датафрейм с описанием и названием фильмов
       - lemmatizer - класс для лемматизации текстов
       - stop_words - стоп-слова
      
       на выходе: 
       - список с предобработанными текстовыми данными
    '''
    
    texts = data.values.tolist()
    new_text = []

    for i in tqdm(range(len(texts))):
        text = texts[i]

        # приводим к нижнему регистру и дропаем стоп-слова
        punct_free = re.sub('[^a-zA-Z]', ' ', str(text).lower())
        punct_free = re.sub(r'\s+', ' ', punct_free)
        stop_free = " ".join([i for i in punct_free.split() if i not in stop_words])
        lemma_text = ' '.join([lemmatizer.lemmatize(word) for word in stop_free.split(' ')])
        new_text.append(lemma_text)
    
    return new_text

In [ ]:
nltk.download("stopwords")

text_overview = preprocessing(movies_metadata["overview"])
text_title = preprocessing(movies_metadata["title"])
text_dir = preprocessing(movies_metadata["director"])
text_genres = preprocessing(movies_metadata["genres"])
text_company = preprocessing(movies_metadata["production_company"])

movies_metadata["overview"] = text_overview
movies_metadata["title"] = text_title 
movies_metadata["director"] = text_dir
movies_metadata["genres"] = text_genres
movies_metadata["production_company"] = text_company

# закидываем предобработанные описания и названия фильмов в общий список
text_prepr = text_overview + text_title + text_dir + text_genres + text_company

# и разбиваем на отдельные слова
text_prepr = [i.split() for i in text_prepr]
text_overview = [i.split() for i in text_overview]
text_title = [i.split() for i in text_title]
text_dir = [i.split() for i in text_dir]
text_genres = [i.split() for i in text_genres]
text_company = [i.split() for i in text_company]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/6835 [00:00<?, ?it/s]

  0%|          | 0/6835 [00:00<?, ?it/s]

  0%|          | 0/6835 [00:00<?, ?it/s]

  0%|          | 0/6835 [00:00<?, ?it/s]

  0%|          | 0/6835 [00:00<?, ?it/s]

In [ ]:
# с помощью word2vec получаем векторизованный словарь
model = Word2Vec(text_prepr, 
                 min_count=2)

model.build_vocab(text_prepr)
words = model.wv.index_to_key
vocab_size = len(words)

print("Vocab size =", vocab_size)

Vocab size = 16575


После обработки всех текстовых данных перейдем к созданию матрицы эмбедингов для каждого фильма

In [ ]:
def embedding_matrix(text, model):
    
    """
       На вход: словарь tokenizer.word_index 
       модель word2vec
        
       На выходе: матрица эмбедингов текста
    """
    
    embedding_matrix = np.zeros((id_film, 100))

    for sentence in text:
        for word in sentence:
            if word in model:
                embedding_matrix[text.index(sentence)] = model[word]

    return embedding_matrix

In [ ]:
id_film = movies_metadata['movieId'].nunique()

# получим матрицы с эмбедингами для всех текстовых признаков
matrix_emb_over = embedding_matrix(text_overview, model.wv)
matrix_emb_title = embedding_matrix(text_title, model.wv)
matrix_emb_dir = embedding_matrix(text_dir, model.wv)
matrix_emb_genres = embedding_matrix(text_genres, model.wv)
matrix_emb_company = embedding_matrix(text_company, model.wv)

# и соединим их
matrix_emb = pd.concat([movies_metadata[['movieId']],
                        pd.DataFrame(matrix_emb_over),
                        pd.DataFrame(matrix_emb_title),
                        pd.DataFrame(matrix_emb_genres)], axis=1)

In [ ]:
matrix_emb.shape

(6835, 301)

In [ ]:
matrix_emb.head(4)

,movieId,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99
0,862,0.495806,1.448255,0.891447,-0.287562,-0.406398,-1.275137,-0.362007,1.161994,-0.403363,-0.079192,-0.419418,-0.609252,-0.084679,-0.299261,-0.014496,-0.274076,1.485301,-0.380626,0.356770,-1.860791,0.680544,1.337196,0.136532,-0.687902,-0.449632,0.494139,-1.582103,-0.201983,0.082339,0.350844,0.366524,0.363220,0.126465,-1.239939,0.093559,1.892473,-0.353858,-0.941660,-0.442541,-2.148511,0.093138,-0.909997,0.749275,0.735510,-0.193813,-1.187146,-0.691451,-0.925867,-0.655156,0.403440,-0.083754,-0.891207,-0.395296,-1.040046,-0.161240,-0.594956,-0.409569,-0.316077,-1.089585,0.505115,0.077344,-0.002151,0.009679,1.125123,-0.081597,0.864145,0.709746,-0.430825,-1.555257,0.013394,-2.220542,-0.524707,0.550566,-0.209737,1.135540,-0.020349,-0.451137,-0.061292,-0.062551,0.293149,-0.482371,1.143046,-0.597678,1.474816,-0.835758,0.400068,0.773842,1.595554,0.859841,0.108987,0.560503,-0.934420,0.812958,-0.476148,0.897804,-0.933257,-0.883437,0.083045,-0.126825,0.489892,0.010273,0.817648,0.634139,0.064504,-0.139594,-1.374062,0.191670,1.151687,-0.186589,-0.471425,-0.087771,-1.131396,-0.106744,-0.003237,0.401782,-0.438085,0.408385,-1.165699,0.457500,-1.060441,0.624667,0.688522,0.641950,-0.279720,-0.245806,0.365756,-0.892511,-0.160167,-0.457349,0.196876,0.340872,0.408380,-0.094786,-0.472401,-0.118263,0.888826,-0.254706,-0.638613,-0.689231,-1.192017,0.102563,-0.546408,0.138354,0.137194,0.170622,-0.456468,-0.654755,-0.071106,0.152520,0.608446,0.198214,-0.485435,-0.088130,-0.413190,-0.453080,0.329939,0.223227,0.287520,-0.346825,-0.083561,-0.129311,0.346390,-0.202848,0.105761,-0.601579,0.487874,-0.243105,0.179237,-0.892699,0.495319,-0.863223,0.446231,0.806208,-0.367043,0.736964,0.318612,0.058537,-0.271030,-0.659621,0.402388,-0.674195,0.251463,-0.322102,1.177188,-0.211923,0.072287,0.019434,0.903923,0.991130,0.475097,0.761710,-0.253769,0.050682,-0.018734,0.784636,0.114753,0.303759,-0.534547,0.138582,0.190755,0.006580,0.050562,0.042159,0.002902,-0.006681,-0.082381,0.000520,0.070367,-0.009425,-0.021532,-0.013027,-0.053693,0.000774,-0.007648,0.019815,-0.026922,0.019735,-0.066364,0.028586,-0.065547,0.035299,0.045478,0.040643,-0.027544,-0.017282,0.010032,-0.046986,0.003749,-0.019555,0.017475,0.019257,0.026409,-0.009899,-0.023559,-0.008478,0.064590,-0.014694,-0.042117,-0.040501,-0.060161,0.000420,-0.022482,0.003914,0.023426,0.015593,-0.023917,-0.032871,-0.017032,-0.002670,0.033919,0.021970,-0.027753,-0.015201,-0.029768,-0.034677,0.007850,0.003210,0.023363,-0.016003,0.003117,0.004139,0.016020,-0.024963,0.005069,-0.036666,0.037161,-0.011343,0.007207,-0.063944,0.032758,-0.065748,0.016840,0.054820,-0.010451,0.043978,0.021965,0.004482,-0.009569,-0.028647,0.030098,-0.038690,0.019087,-0.017193,0.080019,-0.015320,0.009922,-0.006024,0.047654,0.058172,0.032042,0.043557,-0.020263,0.005618,0.001167,0.046387,0.005253,0.004573,-0.037010,0.013595,0.018958
1,8844,-0.397078,1.018687,0.421171,0.370865,-0.321981,-1.651462,0.351163,1.779780,-0.170813,-0.962608,-0.103198,-1.730743,0.264531,0.335708,0.126408,-0.713016,0.396403,-1.411154,0.226196,-0.948250,0.486397,0.393592,0.871127,-0.233695,-0.438025,0.467180,-1.345135,-0

**4 Напишем аналог функции train_test_split для пользователя и его предпочтений**


Подаем на вход функции юзера и получаем два списка фильмов: на тренировочном будем учить, а фильмы из тестового набора искать в предсказаниях

In [ ]:
def train_test_split_films(dataframe, user_list, size=0.7):

  '''
     на вход подается:
     - dataframe - название датафрейма с рейтингами
     - user_list - список пользователей
     - size - пропорция разделения

     на выходе для всех пользователей получаем один словарь, где
     ключ - id зрителя
     список 1 - айдишники train фильмов
     список 2 - оценки train фильмов
     список 3 - айдишники test фильмов
     список 4 - оценки test фильмов
  '''

  user_tts_data_dict = {}

  # итерируемся по всем пользователям
  for user in user_list:

    # выбираю данные по нужному юзеру и перемешиваю
    temp_df = dataframe[dataframe['userId'] == user] \
              .sample(dataframe[dataframe['userId'] == user].shape[0]) \
              [['movieId', 'rating']]

    # отфильтровываем пользователей с количеством просмотров < 10
    if temp_df.shape[0] < 10:
      continue

    # вычисляю длину тренировочного набора для нужного юзера
    shape = int(temp_df.shape[0]*size)

    # сразу записываем в словарь четыре вектора
    user_tts_data_dict[user] = [temp_df.iloc[:shape]['movieId'].tolist(),
                                temp_df.iloc[:shape]['rating'].tolist(),
                                temp_df.iloc[shape:]['movieId'].tolist(),
                                temp_df.iloc[shape:]['rating'].tolist()]

  return user_tts_data_dict

In [ ]:
# пользователей слишком много, мы (в силу отсутствия мощностей) не сможем 
# построить предсказания по всем, но можно выбрать сэмпл
len(ratings['userId'].unique())

258025

In [ ]:
# вторым параметром на вход можно подать любой список id пользователей
splitted_data = train_test_split_films(ratings, ratings['userId'].unique()[:1000], 0.7)

Мы получили словарь, в котором для каждого пользователя, который посмотрел как минимум 10 фильмов, есть тренировочный набор фильмов и оценки к ним + тестовый набор фильмов и оценки к ним

**5 Для каждого пользователя по train-фильмам находим средний вектор предпочтений**

In [ ]:
def get_avg_weightened_vec(dict_name, movies_emb_df):

  '''
     на вход подается:
     - dict_name - название словаря с данными после сплита 
     - movies_emb_df - название эмбединг-датасета с фильмами

     далее мы находим все эмбединг-вектора для train фильмов, домножаем
     каждый на оценку от зрителя и находим средний вектор для каждого зрителя

     на выходе:
     - словарь с ключами - айди пользователей и значениями - средним взвешенным 
     вектором по просмотренным фильмам в train-наборе по каждому пользователю
  '''

  user_vec_data_dict = {}

  # итерируемся по всем пользователям
  for key in dict_name.keys():

    # находим все вектора фильмов из train-выборки в эмбединг-матрице
    # и джойним с их рейтингами от выбранного пользователя
    temp_df = movies_emb_df \
              .loc[movies_emb_df['movieId'].isin(dict_name[key][0])] \
              .merge(pd.DataFrame(dict_name[key][0], dict_name[key][1], 
                                  columns=['movieId'])
                                  .reset_index()
                                  .rename(columns={'index':'rating'}), on='movieId')

    # домножаем эмбединг-вектора на веса-оценки и вычисляем средний вектор
    avg_weightened_vec = list(temp_df \
                              .drop(['movieId'], axis=1)
                              .mul(temp_df['rating'], axis=0)
                              .drop(['rating'], axis=1)
                              .sum()
                              .mul(1 / temp_df['rating'].sum(), axis=0))
    
    # все вектора записываем в словарь под ключ-айди пользователя
    user_vec_data_dict[key] = avg_weightened_vec

  return user_vec_data_dict

In [ ]:
# получим матрицу пользователей
res_vec_dict = get_avg_weightened_vec(splitted_data, matrix_emb)

In [ ]:
# преобразуем к табличному виду
res_vec_df = pd.DataFrame.from_dict(res_vec_dict, orient='index').reset_index().rename(columns={'index':'Id'})

In [ ]:
res_vec_df.head(4)

,Id,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299
0,1,-0.018368,0.238340,0.160970,-0.048378,-0.029982,-0.525233,-0.017152,0.290651,-0.040276,-0.131653,-0.011333,-0.340068,-0.078016,-0.090914,0.247114,-0.082978,0.095844,-0.433282,0.118622,-0.471143,0.228818,0.215416,0.290279,-0.065565,-0.076284,0.167153,-0.266771,-0.024868,-0.125681,0.041959,0.069214,0.207843,0.073805,-0.052684,-0.101087,0.274677,-0.133384,-0.252315,-0.272264,-0.443344,-0.004165,-0.174596,-0.026027,0.098561,0.043315,-0.237325,-0.248838,-0.047629,0.133941,0.151715,0.037596,-0.161846,-0.005012,-0.182680,-0.096765,0.092055,-0.006946,0.104290,-0.026857,0.012767,-0.009293,0.106715,-0.104370,-0.005028,-0.170410,0.201428,-0.109255,0.129799,-0.228212,0.233809,-0.338057,0.151667,0.274829,-0.008511,0.208413,0.082469,0.026558,-0.148393,-0.176963,0.140897,-0.180698,0.137688,-0.173312,0.367784,-0.007557,-0.033438,0.008750,0.267498,0.469679,0.083572,0.362368,-0.042343,0.064512,-0.000737,0.318336,0.054719,0.143027,-0.201326,0.111604,0.033874,-0.000222,0.224015,0.165439,0.017250,-0.035996,-0.367892,0.043193,0.309592,-0.057039,-0.127707,-0.022408,-0.304493,-0.026002,-0.002403,0.103887,-0.121574,0.102605,-0.323664,0.113780,-0.289435,0.165107,0.184499,0.176453,-0.071737,-0.061592,0.098309,-0.240653,-0.042627,-0.120824,0.053324,0.086220,0.104420,-0.019012,-0.122805,-0.037583,0.241852,-0.058859,-0.171493,-0.184630,-0.320249,0.021461,-0.144045,0.028968,0.035646,0.049943,-0.120794,-0.175937,-0.016425,0.040301,0.160879,0.055661,-0.133455,-0.026928,-0.120360,-0.128157,0.083705,0.060650,0.081572,-0.099538,-0.015363,-0.031497,0.097406,-0.060932,0.034006,-0.156772,0.132635,-0.062687,0.055493,-0.232643,0.136869,-0.236223,0.118554,0.216362,-0.094638,0.201460,0.085325,0.017303,-0.074208,-0.171397,0.111225,-0.177739,0.069762,-0.082575,0.319370,-0.050524,0.014621,0.012430,0.241501,0.267245,0.126528,0.215602,-0.056507,0.020750,-0.000355,0.221984,0.030653,0.085440,-0.149700,0.031286,0.054642,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,4,0.003233,0.278149,0.201571,0.023384,-0.046091,-0.449282,0.050347,0.377777,-0.073783,-0.153650,-0.026814,-0.379469,-0.036733,0.000533,0.131921,-0.142637,0.134974,-0.385870,0.144293,-0.354773,0.211509,0.225647,0.212996,-0.087520,-0.081230,0.115008,-0.296731,-0.047063,-0.140017,0.063594,0.108397,0.123986,-0.016446,-0.155984,-0.042347,0.295925,-0.080504,-0.210464,-0.229369,-0.393429,0.029740,-0.174815,0.045280,0.053950,0.065985,-0.156092,-0.214526,-0.033614,0.051409,0.200325,0.070125,-0.161066,-0.031244,-0.147683,-0.152762,0.095390,0.076561,0.095246,-0.112723,-0.019074,-0.034949,0.112374,-0.0

Теперь у нас для каждого пользователя готов вектор его предпочтений, который можно сравнить с фильмами и порекомендовать наиболее подходящие

**6 Найдем топ близких фильмов, которые пользователь еще не смотрел**

Мы будем использовать косинусное расстояние между вектором предпочтений пользователя и векторами фильмов. Косинусное расстояние – это мера похожести двух векторов. Скалярное произведение векторов и косинус угла θ между ними связаны следующим соотношением: a * b = ||a|| ||b|| cosθ. Имея два вектора A и B, получаем косинусное расстояние – cos(θ)

In [ ]:
def get_closest_films(usr_avg_vec_df, user_id, films_emb_mtrx, user_data_dict, top_n=15):

  '''
     на вход подается:
     - usr_avg_vec_df - матрица с train-предпочтениями пользователя
     - user_id - айди пользователя, для которого мы хотим получить рекомендации
     - films_emb_mtrx - эмбединг матрица фильмов
     - splitted_data_dict - словарь с train_test data для всех пользователей
     - top_n - количество рекомендуемых фильмов

     на выходе:
     - список наиболее подходящих непросмотренных фильмов
  '''

  scores = {}

  # достаем средний вектор предпочтений пользователя
  user_avg_vec = usr_avg_vec_df[usr_avg_vec_df['Id'] == user_id] \
                                .drop('Id', axis=1) \
                                .values \
                                .flatten() \
                                .tolist()

  # итерируемся по списку фильмов и находим косинусное расстояние между
  # средним вектором пользователя и каждым фильмом
  for film in films_emb_mtrx['movieId']:

    film_vec = films_emb_mtrx[films_emb_mtrx['movieId'] == film] \
                              .drop('movieId', axis=1) \
                              .values \
                              .flatten() \
                              .tolist()

    # промежуточные результаты записываем в словарь
    scores[film] = 1 - spatial.distance.cosine(user_avg_vec, film_vec)

  # удаляем из словаря фильмы, которые пользователь уже смотрел (в train-наборе)
  for viewed_film in user_data_dict[user_id][0]:
    del scores[viewed_film]

  # возвращаем топ фильмов в порядке убывания близости к вектору пользователя
  # записываем их в пятый список под айди пользователя в словаре с train-test-data
  user_data_dict[user_id].append(heapq.nlargest(top_n, scores, key=scores.get))

In [ ]:
# дополним словарь предсказаниями по каждому пользователю
for i, user in enumerate(splitted_data.keys()):
  print('Running:', i + 1, '/', len(splitted_data))
  get_closest_films(res_vec_df, user, matrix_emb, splitted_data)

Running: 1 / 544


/usr/local/lib/python3.7/dist-packages/scipy/spatial/distance.py:720: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


Running: 2 / 544
Running: 3 / 544
Running: 4 / 544
Running: 5 / 544
Running: 6 / 544
Running: 7 / 544
Running: 8 / 544
Running: 9 / 544
Running: 10 / 544
Running: 11 / 544
Running: 12 / 544
Running: 13 / 544
Running: 14 / 544
Running: 15 / 544
Running: 16 / 544
Running: 17 / 544
Running: 18 / 544
Running: 19 / 544
Running: 20 / 544
Running: 21 / 544
Running: 22 / 544
Running: 23 / 544
Running: 24 / 544
Running: 25 / 544
Running: 26 / 544
Running: 27 / 544
Running: 28 / 544
Running: 29 / 544
Running: 30 / 544
Running: 31 / 544
Running: 32 / 544
Running: 33 / 544
Running: 34 / 544
Running: 35 / 544
Running: 36 / 544
Running: 37 / 544
Running: 38 / 544
Running: 39 / 544
Running: 40 / 544
Running: 41 / 544
Running: 42 / 544
Running: 43 / 544
Running: 44 / 544
Running: 45 / 544
Running: 46 / 544
Running: 47 / 544
Running: 48 / 544
Running: 49 / 544
Running: 50 / 544
Running: 51 / 544
Running: 52 / 544
Running: 53 / 544
Running: 54 / 544
Running: 55 / 544
Running: 56 / 544
Running: 57 / 544


Мы получили топ n предсказаний для каждого пользователя, теперь можно переходить к оценки результатов по основным метрикам

**7 Напишем функции для метрик и оцениваем алгоритм**

Precision можно интепретировать как долю релевантных рекомендаций, мы дополнительно считаем среднее по всем пользователям

Precision = TP / (TP + FP), где TP - фильмы из тестовой выборки, совпавшие с рекомендациями, а TP + FP - это все рекомендованные объекты, то есть K - количество предсказанных объектов

In [ ]:
def mean_precision(user_data_dict):

  '''
     на вход подается:
     - user_data_dict - cловарь с данными о пользователях
     (нас интересует список с индексом 2 - test фильмы и с индексом 4 - рекомендации)

     k берется таким, для которого мы рассчитывали рекомендации (длина рекомендаций)

     на выходе:
     средняя метрика precision@k по всем пользователям
  '''

  sum_precision = 0

  # вычисляем метрику по каждому пользователю
  for user in user_data_dict.keys():
    TP = len(set(user_data_dict[user][4]).intersection(set(user_data_dict[user][2])))
    k = len(user_data_dict[user][4])

  # находим среднее по пользователям
  return sum_precision / len(user_data_dict)

Recall интерпретируется как доля релевантных объектов, попавших в рекомендации, мы дополнительно считаем среднее по всем пользователям

Recall = TP / (TP + FN), где TP - фильмы из тестовой выборки, совпавшие с рекомендациями, а TP + FN - это все объекты из тестовой выборки

In [ ]:
def mean_recall(user_data_dict):

  '''
     на вход подается:
     - user_data_dict - cловарь с данными о пользователях
     (нас интересует список с индексом 2 - test фильмы и с индексом 4 - рекомендации)

     k берется таким, для которого мы рассчитывали рекомендации (длина рекомендаций)

     на выходе:
     средняя метрика recall@k по всем пользователям
  '''

  sum_recall = 0

  # вычисляем метрику по каждому пользователю
  for user in user_data_dict.keys():
    TP = len(set(user_data_dict[user][4]).intersection(set(user_data_dict[user][2])))
    TP_FN = len(user_data_dict[user][2])
    
    sum_recall += TP / TP_FN

  # находим среднее по пользователям
  return sum_recall / len(user_data_dict)

Mean Reciprocal Rank - средний обратный ранк первого правильного вхождения рекомендации в тестовые данные

In [ ]:
def MRR(user_data_dict):

  '''
     RR - обратный ранг, 1 / индеск первого вхождения рекомендации в тест

     на вход подается:
     - user_data_dict - cловарь с данными о пользователях
     (нас интересует список с индексом 2 - test фильмы и с индексом 4 - рекомендации)

     k берется таким, для которого мы рассчитывали рекомендации (длина рекомендаций)

     на выходе:
     метрика MRR@k по всем пользователям
  '''

  sum_rr = 0

  # находим порядковый номер первого вхождения правильной рекомендации в тест
  for user in user_data_dict.keys():
    rank = [1 + splitted_data[user][4].index(x) for x in \
            set(splitted_data[user][4]).intersection(set(splitted_data[user][2]))]
    
    # находим обратный ранк, если попадания нет, то зануляем
    if len(rank) == 0:
      rr = 0
    else:
      rr = 1 / min(rank)

    sum_rr += rr

  # находим среднее по пользователям
  return sum_rr / len(user_data_dict)

Average Precision дополнительно вознаграждает нас за предварительну загрузку рекомендаций, которые будут правильными, то есть здесь, в отличие от Precision, учитывается порядок выдаваемых рекомендаций и их точность отдельно. MAP - это среднее AP по всем пользователям

In [ ]:
def mean_average_precision_at_k(user_data_dict):

  '''
     Mean Average Precision - средняя точность по пользователям

     на вход подается:
     - user_data_dict - cловарь с данными о пользователях
     (нас интересует список с индексом 2 - test фильмы и с индексом 4 - рекомендации)

     k берется таким, для которого мы рассчитывали рекомендации (длина рекомендаций)

     на выходе:
     метрика MAP@k по всем пользователям
  '''

  sum_ap = 0

  # итерируемся по пользователям
  for user in user_data_dict.keys():

    num_hits = 0
    score = 0

    for i, p in enumerate(user_data_dict[user][4]):

      if p in user_data_dict[user][2] and p not in user_data_dict[user][4][:i]:
        num_hits += 1
        score += num_hits / (i + 1)

    sum_ap += score / min(len(user_data_dict[user][2]), len(user_data_dict[user][4]))

  # находим среднее по пользователям
  return sum_ap / len(user_data_dict)

**Результат:**

In [ ]:
print('Mean Precision: ', round(mean_precision(splitted_data), 6))
print('Mean Recall: ', round(mean_recall(splitted_data), 6))
print('Mean Reciprocal Rank:', round(MRR(splitted_data), 6))
print('Mean Average Precision at K:', round(mean_average_precision_at_k(splitted_data), 6))

Mean Precision:  0.0
Mean Recall:  0.002933
Mean Reciprocal Rank: 0.007349
Mean Average Precision at K: 0.000636
